In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ml.models.ensemble import Ensemble

from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

import ray


import time
import pandas as pd
import pyarrow.fs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# rayer.get_global_cluster()
rayer.get_local_cluster()

<module 'ray' from '/home/vagrant/miniconda3/envs/automl/lib/python3.8/site-packages/ray/__init__.py'>

In [4]:
def use_tokamat_ds():
    df = helper.get_tokamat_dataset()
    df = df.reset_index()

    df = common.label_encode(df)
    df = df.fillna(-1)

    potential_targets = ['WTOT', 'WTH', 'PLTH']
    y = df[potential_targets[0]]

    X = df[df.columns[~df.columns.isin(potential_targets)]]
    X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    return train_test_split(X, y, test_size=0.33)


def use_covid_ds():
    df_X, df_y = helper.get_covid_dataset()
    df_X = df_X.drop(['location'], axis = 1)

    return train_test_split(df_X, df_y, test_size=0.33)


def use_transaction_predictions_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)



def use_house_pricing_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)


    df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
    df_y = ds_train['SalePrice']

    return train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)


In [5]:
X_train, X_test, y_train, y_test = use_tokamat_ds()

/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai_0.2.0/notebooks/ensemble/../../utils


In [5]:
r2_scoring = make_scorer(score_func=r2_score, greater_is_better=False)

In [ ]:
ens_mdl = Ensemble(   
                                xgb_objective='binary:logistic',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='binary',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='classification',
                                score_func=None,
                                metric_func=None,
                                list_base_models=[],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [6]:
# list_base_models = ['briskbagging', 'briskknn', 'briskxgboost', 'slugxgboost', 'sluglgbm','slugrf']
ens_mdl = Ensemble(   
                                xgb_objective='count:poisson',  # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]
                                lgbm_objective='poisson',    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
                                pred_class='regression',
                                score_func=None,
                                metric_func=None,
                                list_base_models=['sluglgbm'],
                                n_trials=10,          ### common param
                                epochs=15,             ### ANN param
                                boosted_round=10,      ### boosting tree param
                                max_depth=30,          ### boosting tree param
                                max_n_estimators=1500, ### rf param
                                n_estimators=30,       ### bagging param, must be > 10 
                                n_neighbors=30,        ### knn param, must be > 5

                                ensemble_n_estimators=30,  ###  must be > 10
                                ensemble_n_trials=10,
                                timeout=None
                 )

In [7]:
ens_mdl.fetch_models_pll(X_train, X_test, y_train, y_test)

(run pid=565190) /home/rayuser/.local/lib/python3.8/site-packages/ray/tune/execution/trial_runner.py:363: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
(run pid=565190)   warnings.warn(


(<ray.tune.trainable.util._Trainable object at 0x7fe734f8f5e0> pid=114072, ip=192.168.0.38) [LightGBM] [Warning] lambda_l1 is set=0.0030112853696505518, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0030112853696505518
(<ray.tune.trainable.util._Trainable object at 0x7fe734f8f5e0> pid=114072, ip=192.168.0.38) [LightGBM] [Warning] bagging_fraction is set=1.0615194071732567e-06, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0615194071732567e-06
(<ray.tune.trainable.util._Trainable object at 0x7fe734f8f5e0> pid=114072, ip=192.168.0.38) [LightGBM] [Warning] lambda_l2 is set=0.01462215265581158, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01462215265581158
(<ray.tune.trainable.util._Trainable object at 0x7fe734f8f5e0> pid=114072, ip=192.168.0.38) [LightGBM] [Warning] learning_rate is set=0.1, eta=0.003754600878564795 will be ignored. Current value: learning_rate=0.1
(<ray.tune.trainable.util._Trainable object at 0x7fe734f8f5e0> pid=114072, ip=19

(run pid=565190) 2022-12-07 07:42:53,736	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f70c712c790>}


(<ray.tune.trainable.util._Trainable object at 0x7f07143f6580> pid=114076, ip=192.168.0.38) [LightGBM] [Warning] lambda_l1 is set=0.05105574379786557, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05105574379786557
(<ray.tune.trainable.util._Trainable object at 0x7f07143f6580> pid=114076, ip=192.168.0.38) [LightGBM] [Warning] bagging_fraction is set=0.0003443471605223412, subsample=1.0 will be ignored. Current value: bagging_fraction=0.0003443471605223412
(<ray.tune.trainable.util._Trainable object at 0x7f07143f6580> pid=114076, ip=192.168.0.38) [LightGBM] [Warning] lambda_l2 is set=0.0022677035501819306, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0022677035501819306
(<ray.tune.trainable.util._Trainable object at 0x7f07143f6580> pid=114076, ip=192.168.0.38) [LightGBM] [Warning] learning_rate is set=0.1, eta=0.05881598895353193 will be ignored. Current value: learning_rate=0.1
(<ray.tune.trainable.util._Trainable object at 0x7f07143f6580> pid=114076, ip=192.1

(run pid=565190) 2022-12-07 07:42:57,601	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f70c712c790>}
(run pid=565190) 2022-12-07 07:43:01,572	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f70c712c790>}
(run pid=565190) 2022-12-07 07:43:05,432	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7f70c712c790>}
(run pid=565190) 2022-12-07 07:43:

[LightGBM] [Warning] lambda_l1 is set=0.0010548463116543745, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0010548463116543745
[LightGBM] [Warning] bagging_fraction is set=0.00038734171249778595, subsample=1.0 will be ignored. Current value: bagging_fraction=0.00038734171249778595
[LightGBM] [Warning] lambda_l2 is set=0.015924724095992313, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.015924724095992313
[LightGBM] [Warning] learning_rate is set=0.1, eta=9.95643411382645e-06 will be ignored. Current value: learning_rate=0.1


In [8]:
len(ens_mdl.base_models)

1

In [37]:
ens_mdl.base_model_scores

[[0.9748435941839838, 0.8480916905778864],
 [1.0, 0.3818028641433171],
 [0.0, 0.0],
 [-1.1015093844593219e+25, -1.0934510136033138e+25],
 [-2.77499537657015, -2.9713618168113363],
 [0.8129225481872692, 0.7571846639298663]]

In [9]:
ens_mdl.base_models[0].gs.best_estimator

LGBMRegressor(bagging_fraction=0.00038734171249778595,
              colsample_bytree=0.000116922711111397,
              drop_rate=0.1694029816105492, eta=9.95643411382645e-06,
              lambda_l1=0.0010548463116543745, lambda_l2=0.015924724095992313,
              max_bin=2048, max_depth=8, min_data_in_bin=5, n_estimators=10,
              objective='poisson')

In [29]:
ens_mdl.ensemble.gs.best_estimator

BaggingRegressor(n_estimators=27)

In [ ]:
ens_mdl.scores

[0.907852251809679, 0.9023357075419954]

In [39]:
ens_mdl.predict(X_test)

array([241357.14285714, 164032.85714286, 126121.42857143, 197900.        ,
        94500.        , 122950.        , 271900.        , 179028.57142857,
       525315.28571429, 104935.71428571, 169964.28571429, 241321.42857143,
       218753.57142857, 124928.57142857, 141285.71428571, 145000.        ,
       232383.57142857,  84214.28571429, 148285.71428571, 149071.42857143,
       169392.85714286, 137500.        , 124557.14285714, 174064.28571429,
       197628.57142857, 161517.85714286, 150953.57142857,  82428.57142857,
       266268.07142857, 143885.71428571, 246229.85714286, 193135.71428571,
       129500.        , 242035.71428571, 267855.64285714, 194921.42857143,
       177570.35714286, 126642.85714286, 203685.71428571, 302335.71428571,
       243500.        , 127428.57142857, 172381.07142857, 284242.07142857,
       304321.42857143, 218815.71428571, 127464.28571429, 133857.14285714,
       176000.        , 106000.        , 284785.71428571, 146142.85714286,
       154678.57142857,  

2022-12-06 22:31:16,565	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-12-06 22:31:32,381	ERROR dataclient.py:323 -- Unrecoverable error in data channel.
